In [1]:
%load_ext autoreload

In [21]:
%autoreload

import os, random, pandas as pd, numpy as np
from sklearn.model_selection import StratifiedKFold
import pickle
import sys
sys.path.append('../..')
import relation_extraction.data.utils as utils
import itertools
# SEED = 1
# np.random.seed(SEED)
# random.seed(SEED)
RESOURCE_PATH = "/data/medg/misc/semeval_2010/medical-data/DDICorpus/pre-processed/extraction/"
def res(path): return os.path.join(RESOURCE_PATH, path)

In [22]:
train_data = open(res('train.txt'))
splitted_data_border1 = utils.split_data_cut_sentence(train_data, border_size=1)

In [23]:
train_data=open(res('train.txt'))
splitted_data_border20 = utils.split_data_cut_sentence(train_data, border_size=20)

In [24]:
train_data = open(res('train.txt'))
splitted_data_border50 = utils.split_data_cut_sentence(train_data, border_size=50)

In [25]:
train_data = open(res('train.txt'))
splitted_data_border_minus1 = utils.split_data_cut_sentence(train_data)

In [26]:
def get_dataframe(splitted_data):
    # now need to figure out how to create the csv file
    sen = splitted_data[0]
    rel = splitted_data[1]
    e1 = splitted_data[2]
    e2 = splitted_data[3]
    data = pd.DataFrame({'sentences': sen, 'relations': rel, 'e1_pos': e1, 'e2_pos': e2}, 
                        columns=['sentences', 'relations', 'e1_pos', 'e2_pos'])
    return data

In [27]:
data_border1 = get_dataframe(splitted_data_border1)
data_border20 = get_dataframe(splitted_data_border20)
data_border50 = get_dataframe(splitted_data_border50)
data_borderminus1 = get_dataframe(splitted_data_border_minus1)

In [28]:
len(data_border50)

18387

In [31]:
test_data = open(res('test.txt'))
test_data = utils.split_data_cut_sentence(test_data, border_size=1)
len(test_data[0])

4185

The closest 1/x to the test/train ratio is 1/5. If we do 5 fold split, and keep 1 of the folds as held out test data, then we have a ratio of 1/4 for dev/train which gives us a 25% split, which is close enough to the test/train split

In [33]:
def pickle_data(data, filename1, filename2, SEED):
    np.random.seed(SEED)
    random.seed(SEED)
    print("seed is", SEED)
    N = len(data)
    K = 5

    splitter = StratifiedKFold(n_splits=K, shuffle=True, random_state=SEED)
    fold_indices = [indices for _, indices in splitter.split(
        data[['sentences', 'e1_pos', 'e2_pos']].values,
        data['relations'].values
    )]

    assert len(fold_indices) == K, "Incorrect number of folds."
    assert len(np.concatenate(fold_indices)) == N, "Folds not comprehensive."

    splits = []
    for fold in range(K):
        test_fold  = fold_indices[fold]
        dev_fold   = fold_indices[(fold + 1) % K]

        non_train_start = fold
        non_train_end   = (fold + 2) % K

        if non_train_start < non_train_end:
            train_fold = np.concatenate(fold_indices[non_train_end:] + fold_indices[:non_train_start])
        else: train_fold = np.concatenate(fold_indices[non_train_end:non_train_start])
        joined = np.concatenate((train_fold, dev_fold, test_fold))

        assert len(joined) == N, (
            "Split not comprehensive for fold {fold}:\n"
            "  len(train_fold): {train}\n"
            "  len(dev_fold): {dev}\n"
            "  len(test_fold): {test}\n"
            "  len(joined): {joined}\n"
            "  N: {N}".format(
                N=N, fold=fold, train=len(train_fold), dev=len(dev_fold), test=len(test_fold), joined=len(joined)
            )
        )

        splits.append((
            data.iloc[train_fold],
            data.iloc[dev_fold],
            data.iloc[test_fold],
        ))

    assert len(splits) == K, "Too few splits"

    #below has the dependency info with the directionalities
    #seed_1_{K}-dep-dir-fold.pkl
    with open(res('pickled-files/' + filename1.format(K=K)), mode='wb') as f: pickle.dump(splits, f)
    with open(res('pickled-files/'+filename2.format(K=K)), mode='wb') as f: pickle.dump(splits, f, protocol=2)

In [35]:
for s in range(1,6):
    seed = str(s)
    pickle_data(data_borderminus1, 'seed_'+seed+'_{K}-fold-border_-1.pkl', 'seed_'+seed+'_{K}-fold-border_-1_py2.pkl', s)
    pickle_data(data_border1, 'seed_'+seed+'_{K}-fold-border_1.pkl', 'seed_'+seed+'_{K}-fold-border_1_py2.pkl', s)
    pickle_data(data_border20, 'seed_'+seed+'_{K}-fold-border_20.pkl', 'seed_'+seed+'_{K}-fold-border_20_py2.pkl', s)
    pickle_data(data_border50, 'seed_'+seed+'_{K}-fold-border_50.pkl', 'seed_'+seed+'_{K}-fold-border_50_py2.pkl', s)

seed is 1
seed is 1
seed is 1
seed is 1
seed is 2
seed is 2
seed is 2
seed is 2
seed is 3
seed is 3
seed is 3
seed is 3
seed is 4
seed is 4
seed is 4
seed is 4
seed is 5
seed is 5
seed is 5
seed is 5
